In [ ]:
# Import the castl and multicore package 
from castl.mcfit import *
from pathlib import Path
import multiprocessing
from tqdm import tqdm
import glob
import os

In [ ]:
# Model parameters
model_directory = '/scratch/hcb98/bones_project/default/MODELS/LOWZ'
model_parm = ['Teff', 'log(g)', 'MH', 'CO', 'log(Kzz)']
model_bounds = [(500, 1600), (3.5, 5), (-2.5, 0.5), (0.1, 0.85), (-1, 10)]
unit = 1

In [ ]:
# MCMC settings
steps = 25000
walkers = 40
stretch = 2
discard = 2500
inter_type = 'total'

In [ ]:
# Defines locking for the multicore
lock = None

def init_lock(l):
    global lock
    lock = l

# Defines the running function for multiple objects
def process_file(temp_file):
    split_file = (temp_file.split('/Observed_Spectra/', 2)[1]).split('.csv', 2)[0] # Gets the object name
    output_file = f'Output/MODEL_{split_file}' # Makes the output file name

    file_path = Path(f'{output_file}.pdf')

    # Make sure the object path doesnt already exist
    if file_path.exists():
        pass
    else: 
        with lock:
            print(f'Started Running Calculations for XXX MODEL: {split_file}')
        mcmcfit(temp_file, output_file, model_directory, model_parm, model_bounds, inter_type, unit, steps, walkers, stretch, discard) # Runs the calculations
        
        with lock:
            print(f'Ended Calculations for XXX MODEL: {split_file}')
        return output_file

if __name__ == "__main__":
    input_file_list = glob.glob('/Observed_Spectra/*')
    lock = multiprocessing.Lock()

    # This is how many objects are ran at once (ie 100 total objects with chunk 3 is ~33 objects)
    chunksize = 3

    # Starts each calculation
    with multiprocessing.Pool(initializer=init_lock, initargs=(lock,)) as pool:
        results = []
        for chunk in tqdm(pool.imap(process_file, input_file_list, chunksize=chunksize), total=len(input_file_list)):
            results.append(chunk)

    for output_file in results:
        print(f'Completed: {output_file}')